# 당뇨병 데이터 회귀 분석 노트북

이 Jupyter Notebook은 `scikit-learn`의 `load_diabetes` 데이터셋을 사용하여 다양한 회귀 모델(선형 회귀, Ridge, Lasso, 의사결정트리, 랜덤 포레스트, 그라디언트 부스팅, XGBoost)을 학습하고 평가하는 과정을 보여줍니다.

---
## 1. 라이브러리 임포트

필요한 라이브러리들을 임포트합니다.

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor # xgboost 라이브러리 필요 (pip install xgboost 또는 conda install xgboost)

---
## 2. 데이터 로드 및 확인

당뇨병 데이터셋을 로드하고 데이터의 구조와 내용을 확인합니다. 이 데이터는 당뇨병 진행에 영향을 미치는 요소들과 1년 뒤의 질병 진행도를 예측하는 데 사용됩니다.

In [ ]:
data = load_diabetes() # bunch 라는 클래스 타입으로 정리해서 준다
# 이상치, 누락치, 정규화까지 다 된 자료를 준다 - pandas, numpy

print("데이터셋의 키:", data.keys())
print("타겟 값 샘플 (상위 10개):")
print(data["target"][:10])
print("특성 데이터 샘플 (상위 10개):")
print(data["data"][:10])
print("\n데이터셋 설명:\n", data["DESCR"])

X = data["data"] # 현재 10개의 특성값
y = data["target"] # 미래값으로 나타나는 것

print("\n특성 데이터 형태 (X.shape):", X.shape) # 442개이고 특성이 10개임
print("타겟 데이터 형태 (y.shape):", y.shape)

---
## 3. 데이터 분할

데이터를 훈련 세트와 테스트 세트로 분할합니다. 일반적으로 훈련 세트로 모델을 학습하고, 테스트 세트로 모델의 성능을 평가합니다.

In [ ]:
# 데이터를 훈련 세트와 테스트 세트로 나눈다
X_train, X_test, y_train, y_test = train_test_split \
(X, y, random_state=1234) # 7.5:2.5 비율로 나뉜다 (기본값)

---
## 4. 선형 회귀 모델

가장 기본적인 회귀 모델인 선형 회귀를 학습하고 훈련 세트와 테스트 세트에서의 성능(결정계수 R-squared)을 평가합니다.

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train) # 학습을 하고
y_pred = model.predict(X_test)
# 선형회귀모델의 score 함수는 결정계수를 반환한다. 1이면 완벽하게 예측한 것이고,
# 0이면 거의 예측 불가한 상태를 의미한다.
print("=== Linear Regression Model ===")
print("훈련셋 결정계수:", model.score(X_train, y_train))
print("테스트셋 결정계수:", model.score(X_test, y_test))

---
## 5. Ridge 회귀 모델

선형 회귀에 L2 정규화를 추가하여 과대적합을 방지하는 Ridge 회귀 모델을 학습하고 평가합니다.

In [ ]:
model = Ridge(alpha=0.1) # alpha 값으로 정규화 강도 조절
model.fit(X_train, y_train) # 학습을 하고
y_pred = model.predict(X_test)
print("=== Ridge Model ===")
print("훈련셋 결정계수:", model.score(X_train, y_train))
print("테스트셋 결정계수:", model.score(X_test, y_test))

---
## 6. Lasso 회귀 모델

선형 회귀에 L1 정규화를 추가하여 불필요한 특성의 계수를 0으로 만들어 특성 선택 효과를 내는 Lasso 회귀 모델을 학습하고 평가합니다.

In [ ]:
model = Lasso(alpha=0.1) # alpha 값으로 정규화 강도 조절
model.fit(X_train, y_train) # 학습을 하고
y_pred = model.predict(X_test)
print("=== Lasso Model ===")
print("훈련셋 결정계수:", model.score(X_train, y_train))
print("테스트셋 결정계수:", model.score(X_test, y_test))

---
## 7. 의사결정트리 회귀 모델

트리 기반 모델인 의사결정트리 회귀 모델을 학습하고 평가합니다. 트리 계열 모델은 과대적합 경향이 있을 수 있습니다.

In [ ]:
# 의사결정트리 -> 회귀 가능, 트리 계열은 언제나 과대적합 상태일 수 있음
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(random_state=0) # 재현성을 위해 random_state 설정
model.fit(X_train, y_train) # 학습을 하고
y_pred = model.predict(X_test)
print("=== DecisionTreeRegressor Model ===")
print("훈련셋 결정계수:", model.score(X_train, y_train))
print("테스트셋 결정계수:", model.score(X_test, y_test)) # 회귀분석에서 score가 결정계수값을 반환하며, 음수면 위험
print("특성의 중요도:", model.feature_importances_)

---
## 8. 랜덤 포레스트 회귀 모델

여러 개의 의사결정트리를 앙상블하여 더 안정적이고 강력한 예측 성능을 제공하는 랜덤 포레스트 회귀 모델을 학습하고 평가합니다.

In [ ]:
# 랜덤포레스트: 의사결정트리 + 업그레이드, 여러 개의 분석기를 함께 사용 - 앙상블
# 트리를 랜덤하게 많이 만들어서 평균값을 구한다. 할 때마다 별도의 트리가 만들어져서 계속 측정치가 달라질 수 있다.
# n_estimators: 만들 트리의 최대 개수
# max_depth: 트리의 최대 깊이 지정
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=0, n_estimators=300, max_depth=3)
model.fit(X_train, y_train) # 학습을 하고
y_pred = model.predict(X_test)
print("=== RandomForestRegressor Model ===")
print("훈련셋 결정계수:", model.score(X_train, y_train))
print("테스트셋 결정계수:", model.score(X_test, y_test)) # 회귀분석에서 score가 결정계수값을 반환하며, 음수면 위험
print("특성의 중요도:", model.feature_importances_)

---
## 9. 그라디언트 부스팅 회귀 모델

약한 학습기들을 순차적으로 학습하고 이전 학습기의 오차를 보정해나가면서 성능을 개선하는 그라디언트 부스팅 회귀 모델을 학습하고 평가합니다.

In [ ]:
# 그라디언트 부스팅: 앙상블 계열, 약한 학습기들을 통해서 학습을 하고 보정 작업을 거쳐서 결과를 찾아낸다.
# sklearn GradientBoostingRegressor, xgboost 라이브러리, LightGBM 등
# learning_rate=0.1 학습률, 머신러닝이 학습하는 속도를 조절한다.
# 너무 높으면: 너무 빨리 학습하다가 최적의 위치를 지나칠 수 있다.
# 너무 낮으면: 아주 천천히 느리게 학습을 한다. 아무리 가도 최저점에 도달하지 못하는 경우도 있다.
# GridSearch: 하이퍼파라미터들을 주면 알아서 테스트를 하면서 적절한 파라미터를 찾아낸다. (오래 걸림)

from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(random_state=0, n_estimators=100, max_depth=3, learning_rate=0.1) # n_estimators 기본값 100으로 변경
model.fit(X_train, y_train) # 학습을 하고
y_pred = model.predict(X_test)
print("=== GradientBoostingRegressor Model ===")
print("훈련셋 결정계수:", model.score(X_train, y_train))
print("테스트셋 결정계수:", model.score(X_test, y_test)) # 회귀분석에서 score가 결정계수값을 반환하며, 음수면 위험
print("특성의 중요도:", model.feature_importances_)

---
## 10. XGBoost 회귀 모델

그라디언트 부스팅의 한 종류로, 뛰어난 성능과 효율성으로 널리 사용되는 XGBoost 회귀 모델을 학습하고 평가합니다.

In [ ]:
# xgboost
from xgboost import XGBRegressor
model = XGBRegressor(random_state=0, n_estimators=100, max_depth=3, learning_rate=0.1) # n_estimators 기본값 100으로 변경
model.fit(X_train, y_train) # 학습을 하고
y_pred = model.predict(X_test)
print("=== XGBRegressor Model ===")
print("훈련셋 결정계수:", model.score(X_train, y_train))
print("테스트셋 결정계수:", model.score(X_test, y_test)) # 회귀분석에서 score가 결정계수값을 반환하며, 음수면 위험
print("특성의 중요도:", model.feature_importances_)